In [1]:
# 데이터 로드 및 전처리
# reuters 데이터셋을 로드하고, 훈련 데이터셋에서 25%를 검증 데이터셋으로 분리합니다. 
# 이때 파라메터 조정을 통해 단어의 개수는 10000개로 제한합니다(num_words=10000).
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from keras.datasets import reuters

In [3]:
# 문제 2-1: reuters 데이터셋을 로드하고, 훈련 데이터셋에서 25%를 검증 데이터셋으로 분리합니다.
(train_data, train_labels), (test_data, test_label) = reuters.load_data(num_words=10000)

# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분리 (검증 데이터셋 25%)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.25, random_state=123)

# 훈련, 검증, 테스트 데이터와 레이블 종류가 몇 개인지 출력합니다.
print(len(train_data))
print(len(val_data))
print(len(test_data))
print(len(set(train_labels)))

6736
2246
2246
46


In [5]:
# 문제 2-2: 텍스트 데이터의 전처리를 위해서 각 데이터셋마다 원-핫 인코딩을 수행합니다.
import numpy as np
def one_hot_encoding(data, dim=10000):
  one_hot_data = np.zeros((len(data), dim))
  for i, d in enumerate(data):
    one_hot_data[i, d] = 1.
  return one_hot_data

x_train = one_hot_encoding(train_data)
x_val = one_hot_encoding(val_data)
x_test = one_hot_encoding(test_data)

print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)

# 레이블 데이터들은 범주형 형태로 to_categorical() 함수를 사용해 변환합니다.
from tensorflow.keras.utils import to_categorical

x_train = to_categorical(x_train)
x_val = to_categorical(x_val)
x_test = to_categorical(x_test)

x_train shape: (6736, 10000)
x_val shape: (2246, 10000)
x_test shape: (2246, 10000)


In [ ]:
# 모델 구성 및 컴파일

# 모델 구성
model = models.Sequential()
model.add(layers.Dense(units=3200, activation='relu', input_shape=(6736, )))
model.
model.add(layers.Dense(400, activation='relu'))
model.add(layers.Dense(46, activation='relu'))

# 컴파일
model.compile(optimizer='adam', loss='mse', metrics = 'mae')